In [1]:
from redis import Redis
from os import environ
redis_connection = Redis('this_redis')

# Redis

Redis is a key-value store. Values can be stored in redis associated with a given key.

In [2]:
redis_connection.set('foo', 42)

True

In [3]:
redis_connection.set('bar', [1,2,3])

True

In [4]:
redis_connection.get('bar')

b'[1, 2, 3]'

In [5]:
import requests
request = requests.get('http://this_api:5000/redis/bar')
print(request.text )

<h1>Your Value</h1><p>bar: [1, 2, 3]</p>


One interesting use case is to have Redis track the evolution of some long running iterative process.

In [6]:
import time

def some_iterative_process():
    time.sleep(1)

In [7]:
some_iterative_process()

In [8]:
count = 0
redis_connection.set('count', count)

True

In [9]:
while count < 30:
    some_iterative_process()
    count = redis_connection.incr('count')

In [10]:
count

30

## Passing Parameters Between Processes

We could also use Redis to pass parameters between procesess.

In [11]:
import numpy as np
import json
model_params = {'C':list(np.logspace(-3,3,7)),
                'penalty':'l1',
                'solver' : 'newton-cg'}

In [12]:
model_params

{'C': [0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

In [13]:
type(model_params)

dict

In [14]:
json.dumps(model_params)

'{"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], "penalty": "l1", "solver": "newton-cg"}'

In [15]:
redis_connection.set('model_params', json.dumps(model_params))

True

## Passing Arrays Between Processes

We could also use Redis to pass arrays between procesess.

In [16]:
A = np.array([[1,1,1],
              [2,2,2],
              [3,3,3]])

In [17]:
n, m = A.shape

In [18]:
encoded_A = A.ravel().tostring()

In [19]:
redis_connection.set('encoded_A', encoded_A)

True

In [20]:
redis_connection.set('A_n', n)
redis_connection.set('A_m', m)

True

In [21]:
def matrix_to_redis(matrix, redis_client, key):
    n, m = matrix.shape
    encoded_mat = matrix.ravel().tostring()
    redis_client.set(key, encoded_mat)
    redis_client.set(key+'_n', n)
    redis_client.set(key+'_m', m)

In [22]:
matrix_to_redis(A, redis_connection, 'new_A')

This could also be done via the `pickle` library.

In [23]:
import pickle
import numpy as np

B = np.random.randint(1,10,9).reshape(3,3)
B_pkl = pickle.dumps(B)

redis_connection.set('B', B_pkl)

True

In [24]:
B

array([[5, 1, 7],
       [4, 6, 1],
       [5, 5, 4]])